In [1]:
from database import DatabaseFacade
from decorators import table, ColumnFactory
from model import Record
from crud import CommandInvoker

In [2]:
db_facade = DatabaseFacade()
db_facade.configure("db_test.db")
connection = db_facade.connect()
invoker = CommandInvoker()

In [3]:
@table("users")
class User(Record):
    @ColumnFactory.create_integer_column("id", primary_key=True)
    def id(self): pass

    @ColumnFactory.create_string_column("name", length=50, not_null=True)
    def name(self): pass

    @ColumnFactory.create_string_column("email", length=100, not_null=True)
    def email(self): pass

    @ColumnFactory.create_boolean_column("is_active", default=True)
    def is_active(self): pass

In [4]:
@table("products")
class Product(Record):
    @ColumnFactory.create_integer_column("id", primary_key=True)
    def id(self): pass

    @ColumnFactory.create_string_column("name", length=100, not_null=True)
    def name(self): pass

    @ColumnFactory.create_float_column("price", not_null=True)
    def price(self): pass

    @ColumnFactory.create_boolean_column("in_stock", default=True)
    def in_stock(self): pass

In [5]:
User.configure(connection, invoker)
User.create_table_in_db(db_facade)

Product.configure(connection, invoker)
Product.create_table_in_db(db_facade)

In [6]:
user1 = User(name="Alice", email="alice@example.com")
user2 = User(name="Bob", email="bob@example.com", is_active=False)
user1.save()
user2.save()

product1 = Product(name="Laptop", price=1200.50)
product2 = Product(name="Mouse", price=25.99, in_stock=False)
product1.save()
product2.save()

In [7]:
print("Select names:")
users_names = User.select("name").execute()
for name in users_names:
    print(str(name))

Select names:
<User (name=Alice)>
<User (name=Bob)>


In [9]:
print("Select everything:")
users = User.select().execute()

print(User.as_table(users))

Select everything:
+------+--------+-------------------+-------------+
|   id | name   | email             |   is_active |
+======+========+===================+=============+
|    1 | Alice  | alice@example.com |           1 |
+------+--------+-------------------+-------------+
|    2 | Bob    | bob@example.com   |           0 |
+------+--------+-------------------+-------------+


In [10]:
products_in_stock = Product.select().where("in_stock = TRUE").execute()

print(Product.as_table(products_in_stock))

+------+--------+---------+------------+
|   id | name   |   price |   in_stock |
+======+========+=========+============+
|    1 | Laptop |  1200.5 |          1 |
+------+--------+---------+------------+


In [11]:
user1.email = "alice@gmail.com"
user1.save()

print("After changing Alice's email:")
users = User.select().execute()

print(User.as_table(users))

After changing Alice's email:
+------+--------+-----------------+-------------+
|   id | name   | email           |   is_active |
+======+========+=================+=============+
|    1 | Alice  | alice@gmail.com |           1 |
+------+--------+-----------------+-------------+
|    2 | Bob    | bob@example.com |           0 |
+------+--------+-----------------+-------------+


In [12]:
user1.delete()

print("After deleting Alice ):")

users = User.select().execute()

print(User.as_table(users))

After deleting Alice ):
+------+--------+-----------------+-------------+
|   id | name   | email           |   is_active |
+======+========+=================+=============+
|    2 | Bob    | bob@example.com |           0 |
+------+--------+-----------------+-------------+
